In [10]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import scipy
import itertools

This script focuses on performing differential gene expression analysis on the data corresponding to different respiratory diseases. The genes that are not satistically significant are filtered out for the next step of analysis and to reduce the size of the network 

In [5]:
path = os.getcwd()+'/normalized data/'
os.listdir(path)

['respiratory_data.csv',
 'covid_data.csv',
 'respiratory_metadata.csv',
 'covid_metadata.csv']

## DEGs Function

In [ ]:
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import gmean
from math import floor
from operator import truediv
from statsmodels.stats.multitest import multipletests
import itertools
import warnings
    
    samples = pd.read_csv(sample_file, sep=',', header=0, index_col=0, dtype=str)
    groups = np.unique(samples['condition'])
    print('Found condition groups', groups, 'at time points', timept)
    out = dict()
    for (i,j) in itertools.product(groups, groups):
        if i==j: continue
        else:
            for t in timept:
                geneexp_i = mapped_omics[samples[(samples['condition']==i) & (samples['time']==t)].index].astype('float64').values
                geneexp_j = mapped_omics[samples[(samples['condition']==j) & (samples['time']==t)].index].astype('float64').values
                if geneexp_i.size>0 and geneexp_j.size>0:
                    geo_mean_i = []
                    geo_mean_j = []
                    if geneexp_i.shape[0] != geneexp_j.shape[0]:
                        print('unequal matrix size')
                        break
                    geneexp_ij = np.append(geneexp_i, geneexp_j, 1)
                    zero_transformed = [np.nan]*geneexp_i.shape[0]
                    for r in range(geneexp_i.shape[0]):
                        if np.count_nonzero(geneexp_i[r]) < floor(0.75*geneexp_i.shape[1]) or np.count_nonzero(geneexp_j[r]) < floor(0.75*geneexp_j.shape[1]):
                            if transform_zero:
                                #print(len(np.nonzero(geneexp_ij[r])), np.nonzero(geneexp_ij[r]))
                                zero_transformed[r] = 'yes'
                                if np.count_nonzero(geneexp_ij[r]) == 0:
                                    add_min = 1
                                else:
                                    add_min = np.min(geneexp_ij[r][np.nonzero(geneexp_ij[r])])/2
                                geneexp_i[r] += add_min
                                geneexp_j[r] += add_min
                            else:
                                geneexp_i[r] = np.array([[np.nan]*geneexp_i.shape[1]])
                                geneexp_j[r] = np.array([[np.nan]*geneexp_j.shape[1]])
                        
                        if combo_func == 'gmean':
                            geo_mean_i.append(gmean([x for x in geneexp_i[r] if x!=0]))
                            geo_mean_j.append(gmean([x for x in geneexp_j[r] if x!=0]))
                        if combo_func == 'mean':
                            geo_mean_i.append(np.mean(geneexp_i[r]))
                            geo_mean_j.append(np.mean(geneexp_j[r]))
                        if combo_func == 'median':
                            geo_mean_i.append(np.median(geneexp_i[r]))
                            geo_mean_j.append(np.median(geneexp_j[r]))
                            
                    fc_vals = list(map(truediv, geo_mean_j, geo_mean_i))
                
                    if log:
                        log2fcval = np.log2(fc_vals)
                    else:
                        log2fcval = np.array([-1/i if i<1 else i for i in fc_vals])
                    
                    stat = np.zeros(log2fcval.size)
                    pval = np.zeros(log2fcval.size)
                    prob = np.zeros(log2fcval.size)
                    for idx in range(log2fcval.size):
                        if test_type == 'u':
                            statistic, pvalue = mannwhitneyu(geneexp_i[idx,:], geneexp_j[idx,:])
                        else:
                            statistic, pvalue = ttest_ind(geneexp_i[idx,:], geneexp_j[idx,:], equal_var = False)
                        stat[idx] = statistic
                        pval[idx] = pvalue
                        if log2fcval[idx]>0: prob[idx] = 1-pval[idx]
                        else: prob[idx] = pval[idx]
                    out[(i,j,t)] = pd.DataFrame(np.vstack([mapped_omics['genes'], prob, log2fcval, pval, stat, zero_transformed]).transpose(), columns=['genes', 'probability', 'log2fc', 'pvalue', 'statistic','zero_transformed'])
                    
                  if np.count_nonzero(out[(i,j,t)].log2fc.isnull()) >= floor(0.4*out[(i,j,t)].shape[0]):
                    warnings.warn('Log fold change is not defined for over 40% of genes for condition set {}.\nConsider transforming zero values'.format((i,j,t)))
    if len(out)==0: print('Warning, no appropriate condition groups found! Ensure at last 1 sample for every condition at every time point of interest.')
    return out

In [11]:
sample_file = path + 'respiratory_metadata.csv'
samples = pd.read_csv(sample_file, sep=',', header=0, index_col=0, dtype=str)
groups = np.unique(samples['condition'])
print('Found condition groups', groups)
out = dict()
for (i,j) in itertools.product(groups, groups):
    if i==j: continue
    else: print(i,j)

Found condition groups ['DNA virus infected patient (Adenovirus, Cytomegalovirus, Ebstein-Barr virus, Herpes Simplex virus)'
 'Dengue virus infected patient' 'Entero/Rhinovirus infected patient'
 'Influenza virus infected patient' 'Metapneumovirus infected patient'
 'healthy_ctrl'
 'other respiratory RNA virus infected patient (Parainfluenza virus and Respiratory Syncytial virus)']
DNA virus infected patient (Adenovirus, Cytomegalovirus, Ebstein-Barr virus, Herpes Simplex virus) Dengue virus infected patient
DNA virus infected patient (Adenovirus, Cytomegalovirus, Ebstein-Barr virus, Herpes Simplex virus) Entero/Rhinovirus infected patient
DNA virus infected patient (Adenovirus, Cytomegalovirus, Ebstein-Barr virus, Herpes Simplex virus) Influenza virus infected patient
DNA virus infected patient (Adenovirus, Cytomegalovirus, Ebstein-Barr virus, Herpes Simplex virus) Metapneumovirus infected patient
DNA virus infected patient (Adenovirus, Cytomegalovirus, Ebstein-Barr virus, Herpes Simp

## GSE157240 - Respiratory Disease